# Programação orientada a objetos

Este notebook é a segunda parte de uma adaptação do tutorial ["Python 3 Tutorial"](https://www.python-course.eu/python3_object_oriented_programming.php), do site [python-course.eu](https://www.python-course.eu).

## Escopo de instância versus escopo de classe

Quando instanciamos um objeto de uma classe, o interpretador Python considera que cada instância tem seu próprio escopo. Todos os atributos definidos a nível de instância (usando `self`) são específicos daquela instância.

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return self.name

In [2]:
joao = Person("João")
print(joao)
maria = Person("Maria")
print(maria)

João
Maria


Quando instanciados, todos os objetos de uma mesma classe possuem os mesmos atributos. No entanto, o aspecto dinâmico de Python permite acrescentar e/ou remover atributos de uma instância específica:

In [3]:
joao.surname = "Gomes"
print(joao.surname)

Gomes


Note que a classe `Person` não sabe como se beneficiar da existência desse novo atributo:

In [4]:
print(joao)

João


In [5]:
del maria.name
print(maria)

AttributeError: 'Person' object has no attribute 'name'

Note que alterar o objeto `maria` não altera o objeto `joao`, nem alterar o objeto `joao` altera seu comportamento, uma vez que a classe não define como usar o novo atributo:

In [6]:
print(joao)

João


In [7]:
print(joao.__dict__)

{'surname': 'Gomes', 'name': 'João'}


In [8]:
print(maria.__dict__)

{}


### Escopo de classe

O escopo complementar ao de uma instância é o escopo de classe. 

Esses atributos são compartilhados por todas as instâncias de uma mesma classe, então uma alteração através de uma instância afeta todas as outras.

Pra isto, devemos acessar estes atributos diretamente através do nome da classe, o que é semânticamente bastante legível:

In [9]:
class Person:
    greeting = "Hello"
    
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    def hello(self):
        print("{}, I'm {} :)".format(Person.greeting, self.name))

In [10]:
capitú = Person("Capitú")
capitú.hello()

Hello, I'm Capitú :)


In [11]:
bentinho = Person("Bentinho")
bentinho.hello()

Hello, I'm Bentinho :)


In [12]:
Person.greeting = "Hey"
capitú.hello()
bentinho.hello()

Hey, I'm Capitú :)
Hey, I'm Bentinho :)


Note que tentar alterar um atributo de classe através de uma instância não funciona, porque o Python interpreta que se está criando um atributo de instância de mesmo nome do atributo de classe.

Como o escopo de instância tem prioridade sobre o escopo de classe, o atributo de classe fica mascarado pelo atributo de instância:

In [13]:
print(capitú.greeting)
capitú.greeting = "Hola"
capitú.hello()
bentinho.hello()
print(capitú.greeting)

Hey
Hey, I'm Capitú :)
Hey, I'm Bentinho :)
Hola


### Don't repeat yourself (DRY)

O princípio DRY (não se repita) é uma das principais ferramentas que um programador tem em mãos para evitar erros por replicação de código.

No exemplo anterior, a o método `hello` da classe `Person` está hard-coded (codificado diretamente na classe). Isto pode ser um problema caso o nome da sua classe venha a ser alterado ainda durante a fase de projeto (dificilmente alteramos o nome de uma classe já tornada pública).

Para evitar isto, recomenda-se o uso do procedimento embutido `type`:

In [14]:
class Person:
    greeting = "Hello"
    
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    def hello(self):
        classname = type(self)
        print("{}, I'm {} :)".format(classname.greeting, self.name))

In [15]:
violeta = Person("Violeta")
violeta.hello()

Hello, I'm Violeta :)


### Escopo de métodos

Ainda que todos os métodos definidos em uma classe sejam definidos no escopo de uma classe, é possível diferenciar o argumento passado para o método pelo interpretador.

#### Método de instância

Um **método de instância**, como temos usado até aqui, recebe como primeiro argumento uma referência à instância. Este tipo de método tem acesso a todos os atributos definidos a nível de instância e de classe.

Note que as chamadas a seguir são equivalentes, mas usamos a primeira mais frequentemente por conveniência de escrita:

In [16]:
violeta.hello()
Person.hello(violeta)

Hello, I'm Violeta :)
Hello, I'm Violeta :)


In [17]:
hello = violeta.hello
hello()

Hello, I'm Violeta :)


In [18]:
hello2 = Person.hello
hello2(violeta)

Hello, I'm Violeta :)


In [19]:
from functools import partial
hello_violeta = partial(Person.hello, violeta)
hello_violeta()

Hello, I'm Violeta :)


#### Método de classe

Um **método de classe** recebe como primeiro argumento uma referência à classe. Assim, um método de classe não tem acesso aos atributos definidos a nível de instância.

Para definir um método como um método de classe, usamos o decorador `@classmethod` e adotamos como convenção nomear o primeiro argumento como `cls`:

In [20]:
class Person:
    greeting = "Hello"
    
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    def hello(self):
        classname = type(self)
        print("{}, I'm {} :)".format(classname.greeting, self.name))
        
    @classmethod
    def make_person(cls, name):
        return cls(name)
    
    @classmethod
    def change_greeting(cls, greeting):
        cls.greeting = greeting

In [21]:
crono = Person.make_person("Crono")

In [22]:
print(crono)

Crono


In [23]:
a = Person

In [24]:
magus = a("Magus")

In [25]:
print(magus)

Magus


Há mais de uma forma possível de chamar um método de classe, mas por convenção adotamos a primeira, mais legível e explícita:

In [26]:
ayala = Person.make_person("Ayala")
ayala.change_greeting("Hey")
ayala.hello()
crono.hello()

Hey, I'm Ayala :)
Hey, I'm Crono :)


**Note que uma instância tem acesso a um método de classe pelo mesmo motivo que tem acesso a um atributo de classe - o escopo da classe é buscado quando algo não é encontrado no escopo da instância ;)**

#### Método estático

Um **método estático** não recebe parâmetro algum. Assim, um método estático não tem acesso a nenhum atributo da instância nem da classe - nem mesmo a outros métodos que não sejam estáticos!

Para definir um método como um método de classe, usamos o decorador `@staticmethod`:

In [27]:
class Person:
    greeting = "Hello"
    
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    def hello(self):
        classname = type(self)
        print("{}, I'm {} :)".format(classname.greeting, self.name))
        
    @classmethod
    def make_person(cls, name):
        return cls(name)
    
    @classmethod
    def change_greeting(cls, greeting):
        cls.greeting = greeting
        
    @staticmethod
    def disclaimer():
        print("A person can be good and smart. People tend to be evil and dumb.")

In [28]:
Person.disclaimer()

A person can be good and smart. People tend to be evil and dumb.


Assim como no caso de métodos de classe, métodos estáticos também podem ser invocados através de atributos, mas por convenção evitamos isso.

### Métodos de classe x método estáticos

Poucas linguagens de programação implementam a variação sutil entre métodos de classe e métodos estáticos. Na verdade, [o Guido é bem sincero que isso foi não foi algo intencional](https://www.webucator.com/blog/2016/05/when-to-use-static-methods-in-python-never/):

> We all know how limited static methods are. (They’re basically an accident — back in the Python 2.2 days when I was inventing new-style classes and descriptors, I meant to implement class methods but at first I didn’t understand them and accidentally implemented static methods first. Then it was too late to remove them and only provide class methods.

Tradução livre:
> Todos nós sabemos o quanto métodos estáticos são limitados. (Eles são basicamente um acidente - na época do Python 2.2 quando eu estava inventando descritores e classes no novo estilo, eu queria implementar métodos de classe mas a princípio eu não os entendi e acidentalmente implementei métodos estáticos primeiro. Daí ficou tarde pra removê-los e deixar apenas métodos de classe.

No entanto, a distinção entre método de classe e estático leva a discussão das (des)vantagens de cada um.

Vamos ilustrar o ponto principal com um exemplo sobre métodos-fábrica. 

#### Métodos-fábrica (factory methods)

Nas definições anteriores da classe `Person`, criamos um método de classe `make_person` que constrói um objeto da classe `Person`. De forma simplista, essa é a definição de um método-fábrica.

Vamos ver como seria se o método `make_person` fosse declarado como um método estático:

In [29]:
class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    @staticmethod
    def make_person(name):
        return Person(name)

In [30]:
Person.make_person("Lavos")

Note que o uso de um método estático força o uso do nome da classe dentro do método. Isto fere o princípio DRY e é um dos motivos pelos quais métodos estáticos devem ser evitados.

Vamos analisar agora um caso em que métodos-fábrica são de fato úteis - quando queremos centralizar a criação de objetos de diferentes tipos que pertencem a uma mesma família:

In [31]:
class Person():
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

Dizemos que vários objetos pertencem a uma mesma família quando eles implementam respeitam um mesmo protocolo (polimorfismo informal), implementam uma mesma interface (polimorfismo formal) ou possuem um mesmo ancestral (herança).

Vamos ver um exemplo simples envolvendo herança, que em Python é explicitada no cabeçalho da classe. A classe `Brazilian` abaixo herda da classe `Person`:

In [32]:
class Brazilian(Person):
    def __str__(self):
        return "Sabe sou brasileiro com muito orgulho com muito amor"

Ainda estudaremos em maiores detalhes o conceito e as implicações de herança, mas em poucas palavras uma classe derivada herda de uma classe base seus atributos e métodos.

Também é possível sobrescrever um método herdado, como fizemos com o método mágico `__str__` no exemplo acima. Veja como um objeto da classe `Brazilian` se comporta:

In [33]:
leo = Brazilian("Leo")

In [34]:
print(leo)

Sabe sou brasileiro com muito orgulho com muito amor


Podemos fazer o mesmo com uma classe `Italian`: 

In [35]:
class Italian(Person):
    def __str__(self):
        return "Pasta"

In [36]:
gianpiero = Italian("Gianpiero")

In [37]:
print(gianpiero)

Pasta


Vamos agora centralizar a criação de objetos dessa família de classes em um método estático da classe-base `Person`:

In [38]:
class Person():
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name
        
    @staticmethod
    def create_person(name, nation=""):
        factory = {"it": Italian, "br": Brazilian}
        try:
            return factory[nation](name)
        except KeyError as error:
            return Person(name)

In [39]:
touraj = Person.create_person("Touraj")

In [40]:
print(touraj)

Touraj


In [41]:
edi = Person.create_person("Edi", "br")
print(edi)

Sabe sou brasileiro com muito orgulho com muito amor


In [42]:
mauro = Person.create_person("Mauro", "it")
print(mauro)

Pasta


Note que novamente precisamos colocar o nome da classe quebrando o princípio DRY. 

No entanto, como em todo caso precisamos decidir entre as subclasses dentro do método, não teríamos como evitar quebrar este princípio.

**Na verdade, há um jeito bem interessante de fazer uma fábrica usando o conceito de módulo como objeto, descrito [neste post](https://www.bnmetrics.com/blog/factory-pattern-in-python3-simple-version). Seja curioso e vá conferir :)**